In [48]:
from collections import namedtuple

from vega_sim.null_service import VegaServiceNull
import vega_sim.proto.vega as vega_protos

In [49]:
WalletConfig = namedtuple("WalletConfig", ["name", "passphrase"])

MM_WALLET = WalletConfig("mm", "mm")

TRADER_1_WALLET = WalletConfig("T1", "pin")

TRADER_2_WALLET = WalletConfig("T2", "pin")

TRADER_A1_WALLET = WalletConfig("A1", "pin")

TRADER_A2_WALLET = WalletConfig("A2", "pin")

TERMINATE_WALLET = WalletConfig("Settle", "pin")

wallets = [MM_WALLET, TRADER_1_WALLET, TRADER_2_WALLET, TRADER_A1_WALLET, TRADER_A2_WALLET, TERMINATE_WALLET]

In [50]:
vega = VegaServiceNull(run_with_console=True, use_full_vega_wallet=False)
vega.start()

INFO:vega_sim.null_service:Running NullChain from vegahome of /var/folders/t8/z7v3pdr57d1_rrxyb2skvdtm0000gn/T/vega-sim-m1xcn82w
INFO:vega_sim.null_service:Launching GraphQL node at port 56540
INFO:vega_sim.null_service:Launching Console at port 56546


In [ ]:
for wallet in wallets:
    vega.create_wallet(wallet.name, passphrase=wallet.passphrase)
vega.forward('10s')

In [ ]:
vega.mint(
    MM_WALLET.name,
    asset="VOTE",
    amount=10000,
)

vega.forward("20s")
vega.create_asset(
    MM_WALLET.name,
    name="tDAI",
    symbol="tDAI",
    decimals=5,
    max_faucet_amount=10e15,  # is this including decimals or not?
)

tdai_id = vega.find_asset_id(symbol="tDAI")

vega.mint(
    MM_WALLET.name,
    tdai_id,
    amount=10e7
)
vega.mint(
    TRADER_1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_A1_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.mint(
    TRADER_A2_WALLET.name,
    tdai_id,
    amount=10e5
)
vega.wait_for_total_catchup()

In [ ]:
# Create a market
vega.create_simple_market(
        market_name="BTC:DAI_Mar22",
        proposal_wallet=MM_WALLET.name,
        settlement_asset_id=tdai_id,
        termination_wallet=TERMINATE_WALLET.name,
        position_decimals=2,
        market_decimals=3,
)

market_id = vega.all_markets()[0].id

vega.submit_simple_liquidity(
    wallet_name=MM_WALLET.name,
    market_id=market_id,
    commitment_amount=600,
    fee=0.05,
    reference_buy="PEGGED_REFERENCE_BEST_BID",
    reference_sell="PEGGED_REFERENCE_BEST_ASK",
    delta_buy=20,
    delta_sell=20,
    is_amendment=False,
)

Using function with raw data from data-node VegaService.all_markets. Be wary if prices/positions are not converted from int form


In [ ]:
# Make market pass opening auction
vega.submit_order(
    trading_wallet=TRADER_A1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=900,
    wait=True,
)
vega.submit_order(
    trading_wallet=TRADER_A1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=1,
    price=990,
    wait=True,
)

# Post limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_A2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=1010,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_A2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=1,
    price=1100,
    wait=True,
)

# Post limit order
buy_order_id = vega.submit_order(
    trading_wallet=TRADER_1_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_BUY",
    volume=50,
    price=1000,
    wait=True,
)

sell_order_id = vega.submit_order(
    trading_wallet=TRADER_2_WALLET.name,
    market_id=market_id,
    order_type="TYPE_LIMIT",
    time_in_force="TIME_IN_FORCE_GTC",
    side="SIDE_SELL",
    volume=50,
    price=1000,
    wait=True,
)

In [ ]:
print("Margin level of Trader 1 (price=1000):", "\n", vega.margin_levels(wallet_name=TRADER_1_WALLET.name, market_id=market_id))
general, margin, bond = vega.party_account(wallet_name=TRADER_1_WALLET.name, asset_id=tdai_id, market_id=market_id)
print("Margin account: ", margin)

Margin level of Trader 1 (price=1000): 
 [MarginLevels(maintenance_margin=5153.1421, search_level=5668.4563, initial_margin=6183.77052, collateral_release_level=13398.16946, party_id='6b5db0d68705c364918bb554bb6c0b432fac90f05b8fc1e2eb677746f9cf9b62', market_id='824aa9591e8ad3867091f3ca74f43e33cde61358a59db5c10e1e8cddb2731b19', asset='30650e540f238e051192ec078fd26bb0f0582544343763c9056c6b92f6c2adf4', timestamp=1638185350259066000)]
Margin account:  6183.77052


In [ ]:
# settle market when trader_1 only has open long positions, trader_2 only has open short positions, 
vega.settle_market(
    TERMINATE_WALLET.name,
    settlement_price = 800,
    market_id = market_id,
)
vega.wait_for_total_catchup()

In [ ]:
position = vega.positions_by_market(
        wallet_name=TRADER_1_WALLET.name, market_id = market_id
    )

In [ ]:
print("Margin level of Trader 1 (price=55):", "\n", vega.margin_levels(wallet_name=TRADER_1_WALLET.name, market_id=market_id))
general, margin, bond = vega.party_account(wallet_name=TRADER_1_WALLET.name, asset_id=tdai_id, market_id=market_id)
print("Margin account: ", margin)
print("General account: ", general)
print("RelisedPnL:", position[0].realised_pnl)
print("UnelisedPnL:",position[0].unrealised_pnl)


Margin level of Trader 1 (price=55): 
 [MarginLevels(maintenance_margin=5153.1421, search_level=5668.4563, initial_margin=6183.77052, collateral_release_level=13398.16946, party_id='6b5db0d68705c364918bb554bb6c0b432fac90f05b8fc1e2eb677746f9cf9b62', market_id='824aa9591e8ad3867091f3ca74f43e33cde61358a59db5c10e1e8cddb2731b19', asset='30650e540f238e051192ec078fd26bb0f0582544343763c9056c6b92f6c2adf4', timestamp=1638185350259066000)]
Margin account:  0.0
General account:  1000000.0


IndexError: list index out of range